# Build custom container for Vertex training

In [ ]:
!pwd

## Load env config

* use the prefix from `00-env-setup`

In [ ]:
PREFIX = 'mabv1'

In [ ]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)

In [ ]:
! gsutil ls $BUCKET_URI

## imports

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

## Container Image Variables

In [ ]:
print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [ ]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

## Local Docker build

Provide a name for your dockerfile and make sure you are authenticated

In [ ]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

### Create Dockerfile

In [ ]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

### Create train image

see [example Dockerfile for GPU](https://github.com/GoogleCloudPlatform/cloudml-samples/blob/main/pytorch/containers/quickstart/mnist/Dockerfile-gpu) jobs in Vertex AI

In [ ]:
gpu_profiling = True # True | False

print(f"gpu_profiling : {gpu_profiling}")

In [ ]:
# TRAIN_BASE_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-11:latest'
# docker pull tensorflow/tensorflow:2.13.0-gpu

if gpu_profiling:
    TRAIN_BASE_IMAGE = 'tensorflow/tensorflow:2.13.0-gpu'
    NVTOP_RUN = 'RUN apt update && apt -y install nvtop'
    # NVTOP_RUN = 'RUN apt-get update && apt-get -y install nvtop'
else:
    TRAIN_BASE_IMAGE = 'python:3.10'
    NVTOP_RUN = None
    
RUN_EXPORT = "RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/"
    
print(f"TRAIN_BASE_IMAGE : {TRAIN_BASE_IMAGE}")
print(f"NVTOP_RUN        : {NVTOP_RUN}")
print(f"RUN_EXPORT       : {RUN_EXPORT}")

In [ ]:
dockerfile = f'''
FROM {TRAIN_BASE_IMAGE}

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

{NVTOP_RUN}

RUN ls $APP_HOME

{RUN_EXPORT}

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]
'''
print(dockerfile)

In [ ]:
with open(f'{DOCKERNAME}', 'w') as f:
    f.write(dockerfile)

### Build Image Locally

In [ ]:
!docker build -t $REMOTE_IMAGE_NAME -f $DOCKERNAME .

### Push container to Registry

In [ ]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

### GPU profiling

> enter these commands in the Vertex interactive terminal:

```bash
sudo apt update
sudo apt -y install nvtop
```

**Finished**